### Outline

A talk given by Jeroen Janssens at NYHackR on May 7 2024, covering Polars and data visualisation. He is writing a book called _Python Polars: The Definitive Guide_, to be released this year.

Dataset will be the bike share dataset from NYC.

In [1]:
%pip install polars
%pip install pyarrow
%pip install hvplot
%pip install geopandas


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import polars as pl
import geopandas as gpd
pl.show_versions()

--------Version info---------
Polars:               0.20.24
Index type:           UInt32
Platform:             macOS-14.4.1-arm64-arm-64bit
Python:               3.11.0rc2 (main, Nov  1 2022, 22:31:31) [Clang 14.0.0 (clang-1400.0.29.102)]

----Optional dependencies----
adbc_driver_manager:  <not installed>
cloudpickle:          <not installed>
connectorx:           <not installed>
deltalake:            <not installed>
fastexcel:            <not installed>
fsspec:               <not installed>
gevent:               <not installed>
hvplot:               0.10.0
matplotlib:           <not installed>
nest_asyncio:         1.6.0
numpy:                1.26.4
openpyxl:             <not installed>
pandas:               2.2.2
pyarrow:              16.0.0
pydantic:             <not installed>
pyiceberg:            <not installed>
pyxlsb:               <not installed>
sqlalchemy:           <not installed>
torch:                <not installed>
xlsx2csv:             <not installed>
xlsxwriter:      

### Polars Expressions

- pl.col() turns the name of the column into an object (an expression) which we can do things with: e.g. rename or do some transform. that's not possible if we just use df.select("COL_NAME") as a raw string. We can also use regex inside pl.col() to select columns with a name matching the regex.
- with_columns() is the action that turns expressions into a column in the output, where the expressions are adding a new column. So select() takes columns from an existing dataframe, and with_columns() is adding new columsn to an existing dataframe.
- in aggregation, we're grouping by an expression of some kind (could be a constructed grouping) and then applying an agg() method to say which expressions we want to aggregate.
- we have expressions that are called by different methods (e.g. select(), with_columns(), etc.): an expression is a _tree of operations_ that describes how to construct a Series (a column).
- expressions are lazy, in general, but we also have LazyFrames: we can use those to lazily evaluate a bunch of transformations. These generally have a large speed benefit.

- Note that, in Polars, there is no concept of an Index as in Pandas. It's more like SQL thinking, as far as I can tell.

### Polars Contexts

- A context refers to the context in which an expression (or exporessions) are to be evaluated. There are three main cases: Select, Filter, GroupBy/Aggregate.
- The select context applies expresssions over columns.
- In the filter context, expressions apply on Boolean values based on a given condition.
- In the GroupBy context, expressions apply to groups: the expressions in the .agg() method will be applied to the groups specified in the group_by() context.

The below screenshot shows the case of a GroupBy, under which we are able to specify a number of expressions to be evaluated for each group. Think of this as the `SELECT Name, Sum(Wages), Sum(Hours) FROM * GROUP BY Name` in a SQL query.

![polars group_by](polars_group_by.png)

### Preparing Data

In [3]:
! ls /Volumes/T7/DATASETS/citibike-march-2024/*.csv

/Volumes/T7/DATASETS/citibike-march-2024/202403-citibike-tripdata_1.csv
/Volumes/T7/DATASETS/citibike-march-2024/202403-citibike-tripdata_2.csv
/Volumes/T7/DATASETS/citibike-march-2024/202403-citibike-tripdata_3.csv


We're going to _glob_ the filenames to read all three of these files in one read_csv command

In [4]:
trips = (
    pl.read_csv(
        "/Volumes/T7/DATASETS/citibike-march-2024/202403-citibike-tripdata_*.csv",
        infer_schema_length=100000,
    )
    .select(
        pl.col("rideable_type").str.split("_").list.get(0).alias("bike_type"),
        pl.col("member_casual").alias("rider_type"),
        pl.col("started_at").str.strptime(pl.Datetime).alias("datetime_start"),
        pl.col("ended_at").str.strptime(pl.Datetime).alias("datetime_end"),
        pl.col("start_station_name").alias("station_start"),
        pl.col("end_station_name").alias("station_end"),
        pl.col("start_lat").alias("lat_start"),
        pl.col("start_lng").alias("lon_start"),
        pl.col("end_lat").alias("lat_end"),
        pl.col("end_lng").alias("lon_end"),
    )
    .with_columns((pl.col("datetime_end") - pl.col("datetime_start")).alias("duration"))
)

In [5]:
trips

bike_type,rider_type,datetime_start,datetime_end,station_start,station_end,lat_start,lon_start,lat_end,lon_end,duration
str,str,datetime[μs],datetime[μs],str,str,f64,f64,f64,f64,duration[μs]
"""electric""","""member""",2024-03-07 19:49:43,2024-03-07 20:20:33,"""48 St & Skillman Ave""","""Kingston Ave & Park Pl""",40.746154,-73.916189,40.67308,-73.94191,30m 50s
"""electric""","""member""",2024-03-15 17:45:30,2024-03-15 17:55:39,"""Liberty St & Broadway""","""Mercer St & Spring St""",40.708859,-74.010232,40.723627,-73.999496,10m 9s
"""electric""","""member""",2024-03-19 18:00:52,2024-03-19 18:07:26,"""W 56 St & 6 Ave""","""E 43 St & Madison Ave""",40.763406,-73.977225,40.753547,-73.978966,6m 34s
"""electric""","""member""",2024-03-05 17:25:30,2024-03-05 17:30:17,"""W 56 St & 6 Ave""","""E 43 St & Madison Ave""",40.763062,-73.977672,40.753547,-73.978966,4m 47s
"""electric""","""member""",2024-03-22 13:18:37,2024-03-22 13:23:24,"""5 Ave & W 126 St""","""Frederick Douglass Blvd & W 11…",40.660947,-73.983043,40.803865,-73.955931,4m 47s
…,…,…,…,…,…,…,…,…,…,…
"""classic""","""member""",2024-03-19 12:23:34,2024-03-19 12:29:24,"""Madison Ave & E 51 St""","""Broadway & W 56 St""",40.75863,-73.97513,40.765265,-73.981923,5m 50s
"""classic""","""member""",2024-03-19 17:50:32,2024-03-19 17:57:29,"""Madison Ave & E 51 St""","""Broadway & W 56 St""",40.75863,-73.97513,40.765265,-73.981923,6m 57s
"""classic""","""member""",2024-03-12 09:17:35,2024-03-12 09:27:14,"""West End Ave & W 94 St""","""W 120 St & Claremont Ave""",40.794165,-73.974124,40.810949,-73.9634,9m 39s


In [6]:
trips.select(pl.col("station_start").n_unique())

station_start
u32
2144


In [7]:
trips.filter(
    ~(
        (pl.col("station_start") == pl.col("station_end"))
        & (pl.col("duration") < 60 * 5)
    )
)

bike_type,rider_type,datetime_start,datetime_end,station_start,station_end,lat_start,lon_start,lat_end,lon_end,duration
str,str,datetime[μs],datetime[μs],str,str,f64,f64,f64,f64,duration[μs]
"""electric""","""member""",2024-03-07 19:49:43,2024-03-07 20:20:33,"""48 St & Skillman Ave""","""Kingston Ave & Park Pl""",40.746154,-73.916189,40.67308,-73.94191,30m 50s
"""electric""","""member""",2024-03-15 17:45:30,2024-03-15 17:55:39,"""Liberty St & Broadway""","""Mercer St & Spring St""",40.708859,-74.010232,40.723627,-73.999496,10m 9s
"""electric""","""member""",2024-03-19 18:00:52,2024-03-19 18:07:26,"""W 56 St & 6 Ave""","""E 43 St & Madison Ave""",40.763406,-73.977225,40.753547,-73.978966,6m 34s
"""electric""","""member""",2024-03-05 17:25:30,2024-03-05 17:30:17,"""W 56 St & 6 Ave""","""E 43 St & Madison Ave""",40.763062,-73.977672,40.753547,-73.978966,4m 47s
"""electric""","""member""",2024-03-22 13:18:37,2024-03-22 13:23:24,"""5 Ave & W 126 St""","""Frederick Douglass Blvd & W 11…",40.660947,-73.983043,40.803865,-73.955931,4m 47s
…,…,…,…,…,…,…,…,…,…,…
"""classic""","""member""",2024-03-19 12:23:34,2024-03-19 12:29:24,"""Madison Ave & E 51 St""","""Broadway & W 56 St""",40.75863,-73.97513,40.765265,-73.981923,5m 50s
"""classic""","""member""",2024-03-19 17:50:32,2024-03-19 17:57:29,"""Madison Ave & E 51 St""","""Broadway & W 56 St""",40.75863,-73.97513,40.765265,-73.981923,6m 57s
"""classic""","""member""",2024-03-12 09:17:35,2024-03-12 09:27:14,"""West End Ave & W 94 St""","""W 120 St & Claremont Ave""",40.794165,-73.974124,40.810949,-73.9634,9m 39s


In [8]:
point_start = gpd.GeoSeries(
    gpd.points_from_xy(trips["lon_start"], trips["lat_start"]), crs="EPSG:4326"
).to_crs("EPSG: 32118")
point_end = gpd.GeoSeries(
    gpd.points_from_xy(trips["lon_end"], trips["lat_end"]), crs="EPSG:4326"
).to_crs("EPSG: 32118")
distance = point_start.distance(point_end)

In [9]:
trips = trips.with_columns(distance=pl.lit(pl.Series(distance / 1000)))

In [10]:
stations = (
    trips.group_by(pl.col("station_start").alias("station"))
    .agg(
        pl.col("lat_start").median().alias("lat"),
        pl.col("lon_start").median().alias("lon"),
    )
    .sort("station")
    .drop_nulls()
)

In [11]:
gdf_stations = gpd.GeoDataFrame(
    stations.to_pandas(),
    geometry=gpd.points_from_xy(stations["lon"], stations["lat"]),
    crs="EPSG:4326",
)

In [12]:
gdf_neighborhoods = gpd.read_file("/Volumes/T7/DATASETS/nyc-neighborhood.geojson")
gdf_neighborhoods

,neighborhood,boroughCode,borough,@id,geometry
0,Allerton,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.84860 40.87167, -73.84582 40.870..."
1,Alley Pond Park,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.74333 40.73888, -73.74371 40.739..."
2,Arden Heights,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-74.16983 40.56108, -74.16982 40.561..."
3,Arlington,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-74.15975 40.64142, -74.15998 40.641..."
4,Arrochar,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-74.06078 40.59319, -74.06079 40.593..."
...,...,...,...,...,...
305,Windsor Terrace,3,Brooklyn,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.98006 40.66075, -73.97988 40.660..."
306,Woodhaven,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.86233 40.69596, -73.85654 40.697..."
307,Woodlawn,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.85947 40.90052, -73.85926 40.900..."
308,Woodrow,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-74.17728 40.54063, -74.17723 40.539..."


In [13]:
stations = pl.from_pandas(
    gdf_stations.sjoin(gdf_neighborhoods).drop("geometry", axis=1)
).select(
    pl.col("station"),
    pl.col("borough"),
    pl.col("neighborhood"),
)
stations.filter(pl.col("neighborhood") == "Sunset Park")

station,borough,neighborhood
str,str,str
"""17 St & 5 Ave""","""Brooklyn""","""Sunset Park"""
"""2 Ave & 32 St""","""Brooklyn""","""Sunset Park"""
"""2 Ave & 36 St""","""Brooklyn""","""Sunset Park"""
"""2 Ave & 37 St""","""Brooklyn""","""Sunset Park"""
"""2 Ave & 43 St""","""Brooklyn""","""Sunset Park"""
…,…,…
"""64 St & 3 Ave""","""Brooklyn""","""Sunset Park"""
"""7 Ave & 40 St""","""Brooklyn""","""Sunset Park"""
"""7 Ave & 41 St""","""Brooklyn""","""Sunset Park"""


In [14]:
with pl.Config(tbl_rows=50):
    print(stations.filter(pl.col("neighborhood") == "Sunset Park"))

shape: (55, 3)
┌───────────────────┬──────────┬──────────────┐
│ station           ┆ borough  ┆ neighborhood │
│ ---               ┆ ---      ┆ ---          │
│ str               ┆ str      ┆ str          │
╞═══════════════════╪══════════╪══════════════╡
│ 17 St & 5 Ave     ┆ Brooklyn ┆ Sunset Park  │
│ 2 Ave & 32 St     ┆ Brooklyn ┆ Sunset Park  │
│ 2 Ave & 36 St     ┆ Brooklyn ┆ Sunset Park  │
│ 2 Ave & 37 St     ┆ Brooklyn ┆ Sunset Park  │
│ 2 Ave & 43 St     ┆ Brooklyn ┆ Sunset Park  │
│ 2 Ave & 44 St     ┆ Brooklyn ┆ Sunset Park  │
│ 21 St & 4 Ave     ┆ Brooklyn ┆ Sunset Park  │
│ 23 St & 3 Ave     ┆ Brooklyn ┆ Sunset Park  │
│ 29 St & 5 Ave     ┆ Brooklyn ┆ Sunset Park  │
│ 3 Ave & 17 St     ┆ Brooklyn ┆ Sunset Park  │
│ 30 St & 4 Ave     ┆ Brooklyn ┆ Sunset Park  │
│ 34 St & 4 Ave     ┆ Brooklyn ┆ Sunset Park  │
│ 34 St & 5 Ave     ┆ Brooklyn ┆ Sunset Park  │
│ 36 St & 3 Ave     ┆ Brooklyn ┆ Sunset Park  │
│ 36 St & 4 Ave     ┆ Brooklyn ┆ Sunset Park  │
│ 4 Ave & 17 St     ┆ Bro

In [15]:
stations.select(pl.all().name.suffix("_start")).filter(pl.col("station_start").str.starts_with("5 Ave"))

station_start,borough_start,neighborhood_start
str,str,str
"""5 Ave & 3 St""","""Brooklyn""","""Park Slope"""
"""5 Ave & 37 St""","""Brooklyn""","""Sunset Park"""
"""5 Ave & 41 St""","""Brooklyn""","""Sunset Park"""
"""5 Ave & 67 St""","""Brooklyn""","""Bay Ridge"""
"""5 Ave & E 103 St""","""Manhattan""","""East Harlem"""
…,…,…
"""5 Ave & E 72 St""","""Manhattan""","""Central Park"""
"""5 Ave & E 78 St""","""Manhattan""","""Central Park"""
"""5 Ave & E 87 St""","""Manhattan""","""Central Park"""


In [16]:
all_trips = (
    trips.join(stations.select(pl.all().name.suffix("_start")), on="station_start")
    .join(stations.select(pl.all().name.suffix("_end")), on="station_end")
    .select(
        pl.col("bike_type").cast(pl.Categorical),
        pl.col("rider_type").cast(pl.Categorical),
        "datetime_start",
        "datetime_end",
        "duration",
        "station_start",
        "station_end",
        "neighborhood_start",
        "neighborhood_end",
        "borough_start",
        "borough_end",
        "lat_start",
        "lon_start",
        "lat_end",
        "lon_end",
        "distance",
    )
)

I was having an issue here with the join, where some stations were being incorrectly joined. 

E.g. 

- 5 Ave & 126 St was bing joined to neighborhood Sunset Park
- 7 Ave & Central Park South was joining to Red Hook and 1 Ave & E 62 St was coming up as FiDi

I'm not sure what fixed it, actually. So I need to keep an eye on future joins and check to see if there's a mistake I made somewhere. Maybe (???) cells were run out of order...

In [17]:
all_trips

bike_type,rider_type,datetime_start,datetime_end,duration,station_start,station_end,neighborhood_start,neighborhood_end,borough_start,borough_end,lat_start,lon_start,lat_end,lon_end,distance
cat,cat,datetime[μs],datetime[μs],duration[μs],str,str,str,str,str,str,f64,f64,f64,f64,f64
"""electric""","""member""",2024-03-07 19:49:43,2024-03-07 20:20:33,30m 50s,"""48 St & Skillman Ave""","""Kingston Ave & Park Pl""","""Sunnyside""","""Crown Heights""","""Queens""","""Brooklyn""",40.746154,-73.916189,40.67308,-73.94191,8.400734
"""electric""","""member""",2024-03-15 17:45:30,2024-03-15 17:55:39,10m 9s,"""Liberty St & Broadway""","""Mercer St & Spring St""","""Financial District""","""SoHo""","""Manhattan""","""Manhattan""",40.708859,-74.010232,40.723627,-73.999496,1.874201
"""electric""","""member""",2024-03-19 18:00:52,2024-03-19 18:07:26,6m 34s,"""W 56 St & 6 Ave""","""E 43 St & Madison Ave""","""Midtown""","""Midtown""","""Manhattan""","""Manhattan""",40.763406,-73.977225,40.753547,-73.978966,1.10469
"""electric""","""member""",2024-03-05 17:25:30,2024-03-05 17:30:17,4m 47s,"""W 56 St & 6 Ave""","""E 43 St & Madison Ave""","""Midtown""","""Midtown""","""Manhattan""","""Manhattan""",40.763062,-73.977672,40.753547,-73.978966,1.062271
"""electric""","""member""",2024-03-22 13:18:37,2024-03-22 13:23:24,4m 47s,"""5 Ave & W 126 St""","""Frederick Douglass Blvd & W 11…","""Harlem""","""Harlem""","""Manhattan""","""Manhattan""",40.660947,-73.983043,40.803865,-73.955931,16.035214
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""classic""","""member""",2024-03-19 12:23:34,2024-03-19 12:29:24,5m 50s,"""Madison Ave & E 51 St""","""Broadway & W 56 St""","""Midtown""","""Midtown""","""Manhattan""","""Manhattan""",40.75863,-73.97513,40.765265,-73.981923,0.933801
"""classic""","""member""",2024-03-19 17:50:32,2024-03-19 17:57:29,6m 57s,"""Madison Ave & E 51 St""","""Broadway & W 56 St""","""Midtown""","""Midtown""","""Manhattan""","""Manhattan""",40.75863,-73.97513,40.765265,-73.981923,0.933801
"""classic""","""member""",2024-03-12 09:17:35,2024-03-12 09:27:14,9m 39s,"""West End Ave & W 94 St""","""W 120 St & Claremont Ave""","""Upper West Side""","""Morningside Heights""","""Manhattan""","""Manhattan""",40.794165,-73.974124,40.810949,-73.9634,2.071929


In [18]:
trips_chunked = (
    all_trips.sort("datetime_start")
    .with_columns(date=pl.col("datetime_start").dt.date().cast(pl.String))
    .partition_by(["date"], as_dict=True, include_key=False)
)

for k, df in trips_chunked.items():
    df.write_parquet(f"/Volumes/T7/DATASETS/citibike-march-2024/biketrips-{k[0]}.parquet")

In [19]:
! ls /Volumes/T7/DATASETS/citibike-march-2024/*.parquet

/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-01.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-02.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-03.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-04.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-05.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-06.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-07.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-08.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-09.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-10.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-11.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-12.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-13.parquet
/Volumes/T7/DATASETS/citibike-march-2024/biketrips-2024-03-14.parquet
/Volumes/T7/DATASETS

In [20]:
trip_data = pl.read_parquet("/Volumes/T7/DATASETS/citibike-march-2024/*.parquet")
trip_data

bike_type,rider_type,datetime_start,datetime_end,duration,station_start,station_end,neighborhood_start,neighborhood_end,borough_start,borough_end,lat_start,lon_start,lat_end,lon_end,distance
cat,cat,datetime[μs],datetime[μs],duration[μs],str,str,str,str,str,str,f64,f64,f64,f64,f64
"""electric""","""member""",2024-03-01 00:00:02,2024-03-01 00:27:39,27m 37s,"""W 30 St & 8 Ave""","""Maiden Ln & Pearl St""","""Chelsea""","""Financial District""","""Manhattan""","""Manhattan""",40.749614,-73.995071,40.707065,-74.007319,4.83703
"""electric""","""member""",2024-03-01 00:00:04,2024-03-01 00:09:29,9m 25s,"""Longwood Ave & Southern Blvd""","""Lincoln Ave & E 138 St""","""Longwood""","""Mott Haven""","""Bronx""","""Bronx""",40.816459,-73.896576,40.810893,-73.927311,2.665806
"""classic""","""member""",2024-03-01 00:00:05,2024-03-01 00:31:02,30m 57s,"""W 30 St & 8 Ave""","""Broadway & Morris St""","""Chelsea""","""Financial District""","""Manhattan""","""Manhattan""",40.749653,-73.995208,40.705945,-74.013219,5.086562
"""electric""","""casual""",2024-03-01 00:00:05,2024-03-01 00:03:34,3m 29s,"""Broadway & W 29 St""","""Broadway & W 25 St""","""Midtown""","""Flatiron District""","""Manhattan""","""Manhattan""",40.746424,-73.988559,40.742869,-73.989186,0.398297
"""electric""","""member""",2024-03-01 00:00:09,2024-03-01 00:11:42,11m 33s,"""DeKalb Ave & Franklin Ave""","""6 St & 7 Ave""","""Bedford-Stuyvesant""","""Park Slope""","""Brooklyn""","""Brooklyn""",40.69067,-73.957559,40.668663,-73.979881,3.087634
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""classic""","""member""",2024-03-31 23:59:54,2024-04-01 00:32:09,32m 15s,"""Pier 40 - Hudson River Park""","""Amsterdam Ave & W 73 St""","""SoHo""","""Upper West Side""","""Manhattan""","""Manhattan""",40.727714,-74.011296,40.779668,-73.98093,6.313617
"""electric""","""member""",2024-03-31 23:59:56,2024-04-01 00:02:20,2m 24s,"""Rivington St & Ridge St""","""Ave D & E 3 St""","""Lower East Side""","""East Village""","""Manhattan""","""Manhattan""",40.718701,-73.983141,40.720828,-73.977932,0.49954
"""electric""","""casual""",2024-03-31 23:59:57,2024-04-01 00:09:13,9m 16s,"""12 Ave & W 40 St""","""Riverside Blvd & W 67 St""","""Hell's Kitchen""","""Upper West Side""","""Manhattan""","""Manhattan""",40.760944,-74.002864,40.777507,-73.988886,2.185411


In [21]:
neighborhood = "Red Hook"

trip_speed = trip_data.filter(pl.col("neighborhood_start") == neighborhood).select(
    pl.col("distance"),
    pl.col("duration").dt.total_seconds() / 3600,  # time in hours. why is total_seconds() better than total_minutes()?
    pl.col("bike_type"),
    pl.col("borough_end"),
)
trip_speed

distance,duration,bike_type,borough_end
f64,f64,cat,str
6.666714,0.364167,"""electric""","""Brooklyn"""
0.400557,0.076944,"""electric""","""Brooklyn"""
2.383887,0.190278,"""electric""","""Brooklyn"""
0.471303,0.058611,"""classic""","""Brooklyn"""
1.56366,0.116111,"""electric""","""Brooklyn"""
…,…,…,…
0.797937,0.053333,"""electric""","""Brooklyn"""
1.904528,0.654167,"""electric""","""Brooklyn"""
3.560974,0.2375,"""electric""","""Brooklyn"""


In [22]:
trip_speed.plot.scatter(x="distance", y="duration", color="bike_type", ylim=(0,2))

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


:Scatter   [distance]   (duration,bike_type)

One of the (not very surprising) takeaways from the above chart is that electric bike journeys take place at a higher speed than the classic bike journeys: blue points generally lie at a lower duration for a given distance than do orange points.

A more interesting observation is that there are a notable number of rides that have a duration of ~0, despite having a duration of up to and over an hour. This can be explained by the presence of round trips, where an individual uses the bike for a period of time and returns ot to the same station as it was picked up at. Because our concept of distance is based on a straight line between the start and end points (and we have no way of measuring the distance travelled 'on the road'), there is not much more we can say about these journeys.

In [23]:
"""
If we want to see all the columns of the DataFrame, subject to some filter on the row values, we don't need to use a select statement.
E.g. let's look at all the trips originating in Red Hook where the ending borough is not Brooklyn.
"""
trip_speed.filter(~pl.col("borough_end").is_in(["Brooklyn"]))

distance,duration,bike_type,borough_end
f64,f64,cat,str
9.819964,0.556389,"""electric""","""Manhattan"""
4.919681,0.336389,"""electric""","""Manhattan"""
5.344785,0.578056,"""electric""","""Manhattan"""
4.456671,0.319444,"""electric""","""Manhattan"""
3.821175,0.534722,"""classic""","""Manhattan"""
…,…,…,…
5.834228,0.956111,"""electric""","""Manhattan"""
5.83414,0.955833,"""electric""","""Manhattan"""
4.805438,0.409722,"""electric""","""Manhattan"""


We might also want to look at the patterns in our data. E.g. what is the hourly pattern of rides through the month, with each hour calculated as a proportion of the daily rides?

In [24]:
"""
Start by condensing counts into hourly data, and then find the hourly count as a share of the daily number. We find that daily number by summing over a window of the day. 
We can also add a couple of columns for future use (e.g. a weekday column).
"""
hourly_data = (
    (
        trip_data.sort("datetime_start")
        .group_by_dynamic("datetime_start", every="1h")
        .agg(pl.len().alias("hourly_rides"))
        .with_columns(
            date=pl.col("datetime_start").dt.date()
        )
    )
    .with_columns(
        daily_share=pl.col("hourly_rides")
        / pl.col("hourly_rides").sum().over(pl.col("datetime_start").dt.day())
    )
    .with_columns(
        # quick check column to validate the daily share total sums to 1
        total=pl.col("daily_share")
        .sum()
        .over(
            pl.col("date")
        ),
        weekday=pl.col("date").dt.weekday(),
    )
)

In [25]:
hourly_data

datetime_start,hourly_rides,date,daily_share,total,weekday
datetime[μs],u32,date,f64,f64,i8
2024-03-01 00:00:00,819,2024-03-01,0.00935,1.0,5
2024-03-01 01:00:00,455,2024-03-01,0.005194,1.0,5
2024-03-01 02:00:00,310,2024-03-01,0.003539,1.0,5
2024-03-01 03:00:00,166,2024-03-01,0.001895,1.0,5
2024-03-01 04:00:00,211,2024-03-01,0.002409,1.0,5
…,…,…,…,…,…
2024-03-31 19:00:00,5356,2024-03-31,0.057562,1.0,7
2024-03-31 20:00:00,3784,2024-03-31,0.040667,1.0,7
2024-03-31 21:00:00,2982,2024-03-31,0.032048,1.0,7


We can use a line plot to view the pattern of rides through the day. 

One of the (admitredly difficult to see) insights from this data is that there appears to be a rough pattern of double spikes on weekdays - 
with hours in the morning and evening (around commute times) showing a higher daily share. This pattern doesn't seem to hold for weekend days.

The week of 3/11 to 3/17 is a good example of this pattern.

In [26]:
hourly_data.plot.line(x="datetime_start", y="daily_share")

:Curve   [datetime_start]   (daily_share)

The total number of rides - separated by type of bike, member status, and start borough - can be found using the following group by, where we use the len() aggregation method to count the number of entries in a given grouping. Note that len() has replaced count() in Polars: coming from SQL syntax, this is a difference. 

NOTE: .len() specifically provides the number of rows in a given context, and doesn't exclude nulls. Using .count("col") gives the number of non-null rows in the provided column, "col". See .len() docs [here](https://docs.pola.rs/py-polars/html/reference/expressions/api/polars.len.html)

In [27]:
trips_type_counts = trip_data.group_by("rider_type", "bike_type", "borough_start").len().sort("borough_start","len")
trips_type_counts.head(8)

rider_type,bike_type,borough_start,len
cat,cat,str,u32
"""casual""","""classic""","""Bronx""",1546
"""member""","""classic""","""Bronx""",7185
"""casual""","""electric""","""Bronx""",16097
"""member""","""electric""","""Bronx""",65478
"""casual""","""classic""","""Brooklyn""",37922
"""casual""","""electric""","""Brooklyn""",80511
"""member""","""classic""","""Brooklyn""",208528
"""member""","""electric""","""Brooklyn""",342958


We can also plot count data by different groups using the bar plot function. The documentation for hvplot (the backend behind the .plot() methods) is a little sparse, but the end plots are really nice: interactive and informative. I don't know that they're ideal for static reports, etc. I think for that kind of thing I still like some of the matplotlib design - it's always an option to send the finished Polars dataframe to Pandas and do the plotting from there.

In [28]:
trip_data.group_by("rider_type", "bike_type").len().plot.bar(x="rider_type", y="len", by="bike_type", ylabel="count", yformatter="%.0f", stacked=True, color=["orange", "green"])

/Users/fmcalpine/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/holoviews/core/data/pandas.py:274: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = reindexed.groupby(cols, sort=False)
/Users/fmcalpine/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/holoviews/core/data/pandas.py:274: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = reindexed.groupby(cols, sort=False)
/Users/fmcalpine/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/holoviews/core/data/pandas.py:274: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of p

:Bars   [rider_type,bike_type]   (len)